# Querying nodes

## Preamble
The code in this section is identical to the code in sections "Introduction" and "Loading" from the previous tutorial. It assumes that you have already downloaded the circuit. If not, take a look to the notebook **01_circuits** (Downloading a circuit).

In [1]:
import bluepysnap
import pandas as pd

circuit_path = "sonata/circuit_sonata.json"
circuit = bluepysnap.Circuit(circuit_path)

## Introduction

This section presents different ways to query data from nodes and edges. Since we just went over node sets, lets put them into the spotlight and see how we can utilize them in queries.

## Queries with node sets
As we know, we can use the plain node set name as a string (e.g., `circuit.nodes.ids('VPL_TC')`) to use a node set for queries. However, this only works for the node sets integrated in the circuit.

Luckily, if we want to use node sets external to the circuit config, we can do so by just passing the `NodeSet` object as a query:

In [2]:
node_sets_0_9 = bluepysnap.node_sets.NodeSets.from_dict({'nodes_0-9': {'node_id': [*range(10)]}})

# circuit.nodes.ids('nodes_0-9')  # This would raise an error: BluepySnapError: Undefined node set: 'nodes_0-9'
circuit.nodes.ids(node_sets_0_9['nodes_0-9'])

CircuitNodeIds([('CorticoThalamic_projections', 0),
            ('CorticoThalamic_projections', 1),
            ('CorticoThalamic_projections', 2),
            ('CorticoThalamic_projections', 3),
            ('CorticoThalamic_projections', 4),
            ('CorticoThalamic_projections', 5),
            ('CorticoThalamic_projections', 6),
            ('CorticoThalamic_projections', 7),
            ('CorticoThalamic_projections', 8),
            ('CorticoThalamic_projections', 9),
            ('MedialLemniscus_projections', 0),
            ('MedialLemniscus_projections', 1),
            ('MedialLemniscus_projections', 2),
            ('MedialLemniscus_projections', 3),
            ('MedialLemniscus_projections', 4),
            ('MedialLemniscus_projections', 5),
            ('MedialLemniscus_projections', 6),
            ('MedialLemniscus_projections', 7),
            ('MedialLemniscus_projections', 8),
            ('MedialLemniscus_projections', 9),
            (           'thalamus_ne

## Querying in general
We can query data based on mtype, etype, node_id, region, layer or any of the properties the nodes / edges have:

In [3]:
circuit.nodes.property_names

{'@dynamics:holding_current',
 '@dynamics:threshold_current',
 'etype',
 'layer',
 'model_template',
 'model_type',
 'morph_class',
 'morphology',
 'mtype',
 'orientation_w',
 'orientation_x',
 'orientation_y',
 'orientation_z',
 'region',
 'rotation_angle_xaxis',
 'rotation_angle_yaxis',
 'rotation_angle_zaxis',
 'synapse_class',
 'x',
 'y',
 'z'}

On top of these, a query can also be based on `node_id` or the `population_type`.

When the query is a `dict` and there is a `list` in the query, it is (usually) considered as an "OR" condition, and the keys of the query are considered as an "AND" condition. E.g.,
```python
circuit.nodes.ids({                    # give me the ids of nodes that
    'mtype': ['VPL_TC', 'VPL_IN']      # have mtype of 'VPL_TC' or 'VPL_IN' and
    'population': 'thalamus_neurons'   # belong to a population 'thalamus_neurons'
})  
```

Let's start with simple examples and work our way up to more complex queries.

## Querying with ids

### Integers as `id`s
There are a few methods to query using ids. The most simple way is just use a single integer or a list of integers:

In [4]:
print(circuit.nodes.ids(1))
print(circuit.nodes.ids([1]))

CircuitNodeIds([('CorticoThalamic_projections', 1),
            ('MedialLemniscus_projections', 1),
            (           'thalamus_neurons', 1)],
           names=['population', 'node_ids'])
CircuitNodeIds([('CorticoThalamic_projections', 1),
            ('MedialLemniscus_projections', 1),
            (           'thalamus_neurons', 1)],
           names=['population', 'node_ids'])


This results in any nodes in any of the populations that have nodes with the given id(s). 

If we want to specify a population, we can do the above query with a `dict` instead:

In [5]:
circuit.nodes.ids({'node_id': 1, 'population': 'thalamus_neurons'})

CircuitNodeIds([('thalamus_neurons', 1)],
           names=['population', 'node_ids'])

### Using `CircuitNodeIds` / `CircuitEdgeIds`

We could also use `CircuitNodeIds` (`CircuitEdgeIds` for edges) to specify the population and the node ids to consider. These objects are the same objects returned from the `ids` function and we wouldn't generally need to create them by hand.

The main things you need to know is that these objects are returned from the `ids` functions and they can be directly passed on to `get` functions.

For the sake of an example, lets create a `CircuitNodeIds` object. There are several methods to instantiate them, but we'll use `from_dict` here:

In [6]:
ids_from_dict = bluepysnap.circuit_ids.CircuitNodeIds.from_dict({'thalamus_neurons': [1]})
print(ids_from_dict)

CircuitNodeIds([('thalamus_neurons', 1)],
           names=['population', 'node_ids'])


As you may see, it's exactly what was returned by the `circuit.nodes.ids` function. Let's use it to do a get query:

In [7]:
result = circuit.nodes.get(ids_from_dict, properties=['layer'])
pd.concat([df for _,df in result])

,,layer
population,node_ids,
thalamus_neurons,1,Rt


### Resolved `ids` in `get` queries
As mentioned, we can use the output of the ids as an argument to the `get` function:
```python
ids = circuit.nodes.ids({'node_id': [1, 2], 'population': 'thalamus_neurons'})
circuit.nodes.get(ids, properties=['layer'])
```
but, better yet, we can just pass the query to the `get` function as the `id`s are resolved internally:

In [8]:
result = circuit.nodes.get({'node_id': [1, 2], 'population': 'thalamus_neurons'}, properties=['layer'])
pd.concat([df for _,df in result])

layer
population       node_ids      
thalamus_neurons 1           Rt
                 2           Rt

## Querying with population type
We can query nodes (or edges) based on their population type as specified in the [SONATA circuit configuration file](https://sonata-extension.readthedocs.io/en/latest/sonata_config.html).

Let's find all the source nodes of projections (i.e., `virtual` nodes):

In [9]:
result = circuit.nodes.get({'population_type': ['virtual']})
pd.concat([df for _,df in result])

model_template model_type
population                  node_ids                          
CorticoThalamic_projections 0                          virtual
                            1                          virtual
                            2                          virtual
                            3                          virtual
                            4                          virtual
...                                             ...        ...
MedialLemniscus_projections 5018                       virtual
                            5019                       virtual
                            5020                       virtual
                            5021                       virtual
                            5022                       virtual

[88443 rows x 2 columns]

## Querying with regex
We can use regex to query data (e.g., to cover "OR" cases) by using the key `$regex` and specify an expression as a `str`. For example, let's search `thalamus_neurons` by `mtypes` that start with `VPL`:



In [10]:
 circuit.nodes['thalamus_neurons'].get({"mtype": {"$regex": "^VPL_.*"}}, properties=['mtype'])

,mtype
node_ids,
5002,VPL_TC
5003,VPL_TC
5004,VPL_TC
5005,VPL_TC
5006,VPL_TC
...,...
100760,VPL_IN
100761,VPL_IN
100762,VPL_IN


## Querying ranges of values

As shown above, we can query data based on discrete values of properties such as node set, layer etc. However, all properties are not made equal. 

When there is a `list` in the query, it is _usually_ considered an "OR" condition. There is one exception and that is numeric properties represented by `float`s. Whenever a property is represented by a `float`, a `list` will need to specify exactly two values: start of a range and end of a range (i.e., minimum and maximum values to consider).

Let's take a look which properties in `'thalamus_neurons'` are floats:

In [11]:
circuit.nodes['thalamus_neurons'].property_dtypes

@dynamics:holding_current       float32
@dynamics:threshold_current     float32
etype                          category
layer                            object
model_template                   object
model_type                       object
morph_class                    category
morphology                     category
mtype                          category
orientation_w                   float32
orientation_x                   float32
orientation_y                   float32
orientation_z                   float32
region                           object
rotation_angle_xaxis            float32
rotation_angle_yaxis            float32
rotation_angle_zaxis            float32
synapse_class                  category
x                               float32
y                               float32
z                               float32
dtype: object

Now, let's take `x`, `y`, `z` and query nodes that inside a 50x50x50 box limited by:
* `100 <= x <= 150`
* `500 <= y <= 550`
* `400 <= z <= 450`

In [12]:
circuit.nodes['thalamus_neurons'].get({
    'x': [100, 150],
    'y': [500, 550],
    'z': [400, 450],    
}, properties=['x', 'y', 'z'])

,x,y,z
node_ids,,,
26101,109.218941,526.994873,433.528503
27832,116.552238,519.171082,414.830933
27993,100.712425,535.644287,401.546112
28285,133.707870,531.594727,444.947723
28377,145.187637,547.493591,403.406708
28378,127.572868,538.761536,422.776123
28380,143.356384,525.541626,409.481781
28381,135.235031,517.312378,428.180695


## Complex (or rather, combined) queries
In some cases, we might want to specify our queries further, which makes them complex but not necessarily complicated.

For example, the issue with compound node sets is that it is always considered an "OR" condition, since it's a list.
What if you wanted to combine two node sets and make it an "AND" condition. For this case, there is a key `$node_set` we must combine with yet another key `$and`:

In [13]:
circuit.nodes.ids({
    '$and': [  # list of queries that are considered as: AND conditions
         {'$node_set': 'thalamus_neurons'},
         {'$node_set': 'Rt_RC'}
]})

CircuitNodeIds([('thalamus_neurons',     0),
            ('thalamus_neurons',     1),
            ('thalamus_neurons',     2),
            ('thalamus_neurons',     3),
            ('thalamus_neurons',     4),
            ('thalamus_neurons',     5),
            ('thalamus_neurons',     6),
            ('thalamus_neurons',     7),
            ('thalamus_neurons',     8),
            ('thalamus_neurons',     9),
            ...
            ('thalamus_neurons', 91247),
            ('thalamus_neurons', 91248),
            ('thalamus_neurons', 91249),
            ('thalamus_neurons', 91250),
            ('thalamus_neurons', 91251),
            ('thalamus_neurons', 91252),
            ('thalamus_neurons', 91253),
            ('thalamus_neurons', 91254),
            ('thalamus_neurons', 91255),
            ('thalamus_neurons', 91256)],
           names=['population', 'node_ids'], length=35567)

This, however, can not be done with node sets external to the circuit.

It may also be that you want to query something as an "OR" condition rather than "AND".
For example querying
```python
circuit.nodes.ids({'mtype': 'VPL_IN', 'region': 'mc2;Rt'})
```
will never return any ids. So how do we make it an "OR" case? That's right, with the help of `$or`:

In [14]:
result = circuit.nodes['thalamus_neurons'].get({
    '$or':[ # same as with $and, except the list is considered as OR condition
        {'mtype': 'VPL_IN'},
        {'region': 'mc2;Rt'}
]}, properties=['mtype', 'region'])

# Let's print all the unique mtype-region pairs:
for pair in result.groupby(['mtype', 'region'], observed=True).groups.keys():
    print(pair)

('Rt_RC', 'mc2;Rt')
('VPL_IN', 'mc0;VPL')
('VPL_IN', 'mc1;VPL')
('VPL_IN', 'mc2;VPL')
('VPL_IN', 'mc3;VPL')
('VPL_IN', 'mc4;VPL')
('VPL_IN', 'mc5;VPL')
('VPL_IN', 'mc6;VPL')


## Conclusion
In this notebook we took a deeper look into how to query data in SNAP using nodes as an example. In the next notebook we'll cover edge queries.